In [2]:
import pandas as pd
import csv
import numpy as np
import os

In [3]:
# Change the values that need to be changed manually here
station_coord_rects = {
    'Xavier': { 
        'Latitude': { 
            'lower': 14.640004,
            'higher': 14.640371
        },
        'Longitude': { 
            'lower': 121.078251,
            'higher': 121.078789
        }
    },
    'Old Comm': { 
        'Latitude': { 
            'lower': 14.636606,
            'higher': 14.636749
        },
        'Longitude': { 
            'lower': 121.078116,
            'higher': 121.078324
        }
    },
    'LST': { 
        'Latitude': { 
            'lower': 14.636115,
            'higher': 14.636302
        },
        'Longitude': { 
            'lower': 121.080644,
            'higher': 121.080738
        }
    },
    'Grade School': { 
        'Latitude': { 
            'lower': 14.634715,
            'higher': 14.634715
        },
        'Longitude': { 
            'lower': 121.076171,
            'higher': 121.076326
        }
    },
    'JSEC': { 
        'Latitude': { 
            'lower': 14.637686,
            'higher': 14.637804
        },
        'Longitude': { 
            'lower': 121.076260,
            'higher': 121.076370
        }
    },
    'Gate 2.5': { 
        'Latitude': { 
            'lower': 14.637874,
            'higher': 14.637927
        },
        'Longitude': { 
            'lower': 121.074848,
            'higher': 121.075020
        }
    },
    'Leong': { 
        'Latitude': { 
            'lower': 14.640724,
            'higher': 14.640864
        },
        'Longitude': { 
            'lower': 121.076177,
            'higher': 121.076395
        }
    },
    
}

# Raw Data
* Combine csv files from each rpi into one data frame
* Pre-process raw data

In [4]:
# Create a list to hold the dataframes
df_list = []

for i in range(3):
    i = i + 1
    folder_path = os.path.join('E-Jeep Data', f"rpi{i}")
    all_files = os.listdir(folder_path)
    
    # Filter out non-CSV files
    csv_files = [f for f in all_files if f.endswith('.csv')]
    
    for csv in csv_files:
        file_path = os.path.join(folder_path, csv)
        file_path
        try:
            # Try reading the file using default UTF-8 encoding
            df = pd.read_csv(file_path)
            df.tail(10)

            df.columns = ['Type', 'Time', 'Lat', 'Long', 'Altitude', 'Humidity', 'Temperature']
            df.drop(columns='Type', inplace=True)

            # Drop rows with null at start and end
            df = df.loc[df.notnull().all(axis=1).argmax():]

            ids = df[df.notnull().all(axis=1)].index
            last_id = ids[-1]

            df = df.loc[:last_id]
            
            # Fix time to PHT
            log_date = os.path.basename(file_path).split('/')[-1]
            df['Time'] = pd.to_datetime(df['Time'])
            df['Time'] = df['Time'].dt.tz_convert('+08:00')
            df['Time'] = pd.to_datetime(df['Time'].dt.strftime(f"{log_date[5:15]} %H:%M:%S"))

            # Add day of week
            df['Day_of_Week'] = df['Time'].dt.day_name()

            # Iterpolate values for NaN GPS values
            df['Lat'] = df['Lat'].astype(float).interpolate(method='linear', limit_direction='both') 
            df['Long'] = df['Long'].astype(float).interpolate(method='linear', limit_direction='both') 
            df['Altitude'] = df['Altitude'].astype(float).interpolate(method='linear', limit_direction='both') 
            
            df_list.append(df)
        except UnicodeDecodeError:
            try:
                # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
                df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
                df_list.append(df)
            except Exception as e:
                print(f"Could not read file {csv} because of error: {e}")
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")

# Concatenate all data into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df

/var/folders/6z/nd6ncchj22330v2z44h3b_jr0000gn/T/ipykernel_3300/1474835611.py:17: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/6z/nd6ncchj22330v2z44h3b_jr0000gn/T/ipykernel_3300/1474835611.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time'])
/var/folders/6z/nd6ncchj22330v2z44h3b_jr0000gn/T/ipykernel_3300/1474835611.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time'])
/var/folders/6z/nd6ncchj22330v2z44h3b_jr0000gn/T/ipykernel_3300/1474835611.py:33: UserWarning: Could not infer format, so each element will be parsed individually, fall

,Time,Lat,Long,Altitude,Humidity,Temperature,Day_of_Week
0,2023-10-23 17:10:42,1438.28232,12104.69905,112.5,64.199997,32.799999,Monday
1,2023-10-23 17:10:43,1438.28099,12104.69771,112.4,64.199997,32.799999,Monday
2,2023-10-23 17:10:44,1438.27832,12104.69598,112.2,64.199997,32.799999,Monday
3,2023-10-23 17:10:45,1438.27604,12104.69410,112.1,81.300003,29.299999,Monday
4,2023-10-23 17:10:46,1438.27355,12104.69198,111.9,81.300003,29.299999,Monday
...,...,...,...,...,...,...,...
567181,2023-11-14 00:56:36,1437.98425,12104.48160,107.0,76.000000,29.799999,Tuesday
567182,2023-11-14 00:56:37,1437.98409,12104.48159,106.6,76.000000,29.799999,Tuesday
567183,2023-11-14 00:56:38,1437.98388,12104.48164,106.1,76.000000,29.799999,Tuesday
567184,2023-11-14 00:56:39,1437.98349,12104.48167,105.6,76.000000,29.799999,Tuesday


In [5]:
# Reformat Lat and Long to be decimal coordinates
def ddmm_mmmm_to_decimal(coord):
    # Assuming coord is in the format 'ddmm.mmmm'
    
    # Extract degrees and decimal minutes
    degrees = coord // 100
    decimal_minutes = coord % 100
    
    # Convert decimal minutes to decimal degrees
    decimal_degrees = degrees + decimal_minutes / 60
    
    return decimal_degrees

df['Latitude'] = df.apply(lambda row: ddmm_mmmm_to_decimal(row['Lat']), axis=1)
df['Longitude'] = df.apply(lambda row: ddmm_mmmm_to_decimal(row['Long']), axis=1)

# Drop old Lat / Long values
df.drop(columns=['Lat', 'Long'], inplace=True)
df

,Time,Altitude,Humidity,Temperature,Day_of_Week,Latitude,Longitude
0,2023-10-23 17:10:42,112.5,64.199997,32.799999,Monday,14.638039,121.078317
1,2023-10-23 17:10:43,112.4,64.199997,32.799999,Monday,14.638016,121.078295
2,2023-10-23 17:10:44,112.2,64.199997,32.799999,Monday,14.637972,121.078266
3,2023-10-23 17:10:45,112.1,81.300003,29.299999,Monday,14.637934,121.078235
4,2023-10-23 17:10:46,111.9,81.300003,29.299999,Monday,14.637892,121.078200
...,...,...,...,...,...,...,...
567181,2023-11-14 00:56:36,107.0,76.000000,29.799999,Tuesday,14.633071,121.074693
567182,2023-11-14 00:56:37,106.6,76.000000,29.799999,Tuesday,14.633068,121.074693
567183,2023-11-14 00:56:38,106.1,76.000000,29.799999,Tuesday,14.633065,121.074694
567184,2023-11-14 00:56:39,105.6,76.000000,29.799999,Tuesday,14.633058,121.074694


In [6]:
# Label stations along the line that the ejeep is in based on coordinates
def coord_to_station(lat, long) -> str:
    stations = station_coord_rects.keys()

    for station in stations:
        lat_bounds, long_bounds = station_coord_rects[station]['Latitude'], station_coord_rects[station]['Longitude']

        if (lat >= lat_bounds['lower'] - 0.0003 and lat <= lat_bounds['higher'] + 0.0003) and (long >= long_bounds['lower'] - 0.0003 and long <= long_bounds['higher'] + 0.0003):
            return station
    else:
        return "En Route"

df['Station'] = df.apply(lambda row: coord_to_station(row['Latitude'], row['Longitude']), axis=1)
df

df.to_csv('Out.csv')


In [7]:
import math

# Function to calculate distance using Haversine formula
def haversine_distance(lat_diff, lon_diff):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude differences from degrees to radians
    lat_diff_rad = math.radians(lat_diff)
    lon_diff_rad = math.radians(lon_diff)

    # Haversine formula
    a = math.sin(lat_diff_rad / 2)**2 + math.cos(0) * math.cos(0) * math.sin(lon_diff_rad / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate distance
    distance = R * c

    return distance * 1000

In [8]:
# Calculate speed (noted as distance since it's by second anyway) based on change in coordinates from the previous
# Helps establish if ejeep is truly within station

df['Lat Diff'] = df['Latitude'].diff()
df['Long Diff'] = df['Longitude'].diff()

df['Distance'] = df.apply(lambda row: haversine_distance(row['Lat Diff'], row['Long Diff']), axis=1)
df['Station'] = df.apply(lambda row: 'En Route' if row['Station'] != 'En Route' and row['Distance'] >= 0.5 else row['Station'], axis=1)

In [9]:
# Add distances (by road) to next station
df['Cum Distance'] = df[::-1].groupby((df['Station'] != 'En Route').cumsum())['Distance'].cumsum()
df.loc[df['Station'] != 'En Route', 'Cum Distance'] = 0

df.dtypes


Time            datetime64[ns]
Altitude               float64
Humidity               float64
Temperature            float64
Day_of_Week             object
Latitude               float64
Longitude              float64
Station                 object
Lat Diff               float64
Long Diff              float64
Distance               float64
Cum Distance           float64
dtype: object

In [10]:
# Add distances (as the bird flies) to next station

def station_centroids() -> dict:
    new_dict = dict()

    new_dict['En Route'] = { 
        'Latitude': None,
        'Longitude': None
    }

    for station in station_coord_rects:
        lat_ave = (station_coord_rects[station]['Latitude']['lower'] + station_coord_rects[station]['Latitude']['higher']) / 2
        long_ave = (station_coord_rects[station]['Longitude']['lower'] + station_coord_rects[station]['Longitude']['higher']) / 2

        new_dict[station] = { 
            'Latitude': lat_ave,
            'Longitude': long_ave
        }
    
    return new_dict

centers = station_centroids()

df['Next Station Lat'] = df.apply(lambda row: centers[row['Station']]['Latitude'], axis=1)
df['Next Station Long'] = df.apply(lambda row: centers[row['Station']]['Longitude'], axis=1)

df['Next Station Lat'] = df['Next Station Lat'].interpolate(method='bfill', limit_direction='backward') 
df['Next Station Long'] = df['Next Station Long'].interpolate(method='bfill', limit_direction='backward') 

df['Abs Distance'] = df.apply(lambda row: haversine_distance(row['Next Station Lat'] - row['Latitude'], row['Next Station Long'] - row['Longitude']), axis=1)

df

,Time,Altitude,Humidity,Temperature,Day_of_Week,Latitude,Longitude,Station,Lat Diff,Long Diff,Distance,Cum Distance,Next Station Lat,Next Station Long,Abs Distance
0,2023-10-23 17:10:42,112.5,64.199997,32.799999,Monday,14.638039,121.078317,En Route,NaN,NaN,NaN,NaN,14.636209,121.080691,333.269806
1,2023-10-23 17:10:43,112.4,64.199997,32.799999,Monday,14.638016,121.078295,En Route,-2.216667e-05,-2.233333e-05,3.498912,465.044503,14.636209,121.080691,333.749333
2,2023-10-23 17:10:44,112.2,64.199997,32.799999,Monday,14.637972,121.078266,En Route,-4.450000e-05,-2.883333e-05,5.896070,461.545591,14.636209,121.080691,333.379761
3,2023-10-23 17:10:45,112.1,81.300003,29.299999,Monday,14.637934,121.078235,En Route,-3.800000e-05,-3.133333e-05,5.476593,455.649522,14.636209,121.080691,333.756833
4,2023-10-23 17:10:46,111.9,81.300003,29.299999,Monday,14.637892,121.078200,En Route,-4.150000e-05,-3.533333e-05,6.060577,450.172928,14.636209,121.080691,334.373291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567181,2023-11-14 00:56:36,107.0,76.000000,29.799999,Tuesday,14.633071,121.074693,En Route,-3.166667e-06,-1.666667e-07,0.352605,2.161636,NaN,NaN,NaN
567182,2023-11-14 00:56:37,106.6,76.000000,29.799999,Tuesday,14.633068,121.074693,En Route,-2.666667e-06,-1.666667e-07,0.297098,1.809032,NaN,NaN,NaN
567183,2023-11-14 00:56:38,106.1,76.000000,29.799999,Tuesday,14.633065,121.074694,En Route,-3.500000e-06,8.333333e-07,0.400061,1.511933,NaN,NaN,NaN
567184,2023-11-14 00:56:39,105.6,76.000000,29.799999,Tuesday,14.633058,121.074694,En Route,-6.500000e-06,5.000000e-07,0.724902,1.111872,NaN,NaN,NaN


In [11]:
# Do percentage of distance remaining to next station

# Calculate the maximum value of C within each group of 'X'
max_distance_per_group = df[df['Station'] == 'En Route'].groupby((df['Station'] != 'En Route').cumsum())['Cum Distance'].transform('max')

# Add column D representing the percentage of each value in column C relative to the maximum value within its group
df['Percent Distance'] = df['Cum Distance'] / max_distance_per_group * 100

df.to_csv('Out.csv')

df.dtypes


Time                 datetime64[ns]
Altitude                    float64
Humidity                    float64
Temperature                 float64
Day_of_Week                  object
Latitude                    float64
Longitude                   float64
Station                      object
Lat Diff                    float64
Long Diff                   float64
Distance                    float64
Cum Distance                float64
Next Station Lat            float64
Next Station Long           float64
Abs Distance                float64
Percent Distance            float64
dtype: object

In [12]:
# Add distance (by road) to the previous station
df['Prev Cum Distance'] = df.groupby((df['Station'] != 'En Route').cumsum())['Distance'].cumsum()
df.loc[df['Station'] != 'En Route', 'Prev Cum Distance'] = 0

df

,Time,Altitude,Humidity,Temperature,Day_of_Week,Latitude,Longitude,Station,Lat Diff,Long Diff,Distance,Cum Distance,Next Station Lat,Next Station Long,Abs Distance,Percent Distance,Prev Cum Distance
0,2023-10-23 17:10:42,112.5,64.199997,32.799999,Monday,14.638039,121.078317,En Route,NaN,NaN,NaN,NaN,14.636209,121.080691,333.269806,NaN,NaN
1,2023-10-23 17:10:43,112.4,64.199997,32.799999,Monday,14.638016,121.078295,En Route,-2.216667e-05,-2.233333e-05,3.498912,465.044503,14.636209,121.080691,333.749333,100.000000,3.498912
2,2023-10-23 17:10:44,112.2,64.199997,32.799999,Monday,14.637972,121.078266,En Route,-4.450000e-05,-2.883333e-05,5.896070,461.545591,14.636209,121.080691,333.379761,99.247618,9.394982
3,2023-10-23 17:10:45,112.1,81.300003,29.299999,Monday,14.637934,121.078235,En Route,-3.800000e-05,-3.133333e-05,5.476593,455.649522,14.636209,121.080691,333.756833,97.979767,14.871575
4,2023-10-23 17:10:46,111.9,81.300003,29.299999,Monday,14.637892,121.078200,En Route,-4.150000e-05,-3.533333e-05,6.060577,450.172928,14.636209,121.080691,334.373291,96.802118,20.932152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567181,2023-11-14 00:56:36,107.0,76.000000,29.799999,Tuesday,14.633071,121.074693,En Route,-3.166667e-06,-1.666667e-07,0.352605,2.161636,NaN,NaN,NaN,0.203152,1062.558539
567182,2023-11-14 00:56:37,106.6,76.000000,29.799999,Tuesday,14.633068,121.074693,En Route,-2.666667e-06,-1.666667e-07,0.297098,1.809032,NaN,NaN,NaN,0.170014,1062.855637
567183,2023-11-14 00:56:38,106.1,76.000000,29.799999,Tuesday,14.633065,121.074694,En Route,-3.500000e-06,8.333333e-07,0.400061,1.511933,NaN,NaN,NaN,0.142092,1063.255699
567184,2023-11-14 00:56:39,105.6,76.000000,29.799999,Tuesday,14.633058,121.074694,En Route,-6.500000e-06,5.000000e-07,0.724902,1.111872,NaN,NaN,NaN,0.104494,1063.980601


In [14]:
# Add distance (as the crow flies) to the previous station
df['Prev Station Lat'] = df.apply(lambda row: centers[row['Station']]['Latitude'], axis=1)
df['Prev Station Long'] = df.apply(lambda row: centers[row['Station']]['Longitude'], axis=1)

df['Prev Station Lat'] = df['Prev Station Lat'].interpolate(method='ffill', limit_direction='forward') 
df['Prev Station Long'] = df['Prev Station Long'].interpolate(method='ffill', limit_direction='forward') 

df['Prev Abs Distance'] = df.apply(lambda row: haversine_distance(row['Prev Station Lat'] - row['Latitude'], row['Prev Station Long'] - row['Longitude']), axis=1)

df

,Time,Altitude,Humidity,Temperature,Day_of_Week,Latitude,Longitude,Station,Lat Diff,Long Diff,Distance,Cum Distance,Next Station Lat,Next Station Long,Abs Distance,Percent Distance,Prev Cum Distance,Prev Station Lat,Prev Station Long,Prev Abs Distance
0,2023-10-23 17:10:42,112.5,64.199997,32.799999,Monday,14.638039,121.078317,En Route,NaN,NaN,NaN,NaN,14.636209,121.080691,333.269806,NaN,NaN,NaN,NaN,NaN
1,2023-10-23 17:10:43,112.4,64.199997,32.799999,Monday,14.638016,121.078295,En Route,-2.216667e-05,-2.233333e-05,3.498912,465.044503,14.636209,121.080691,333.749333,100.000000,3.498912,NaN,NaN,NaN
2,2023-10-23 17:10:44,112.2,64.199997,32.799999,Monday,14.637972,121.078266,En Route,-4.450000e-05,-2.883333e-05,5.896070,461.545591,14.636209,121.080691,333.379761,99.247618,9.394982,NaN,NaN,NaN
3,2023-10-23 17:10:45,112.1,81.300003,29.299999,Monday,14.637934,121.078235,En Route,-3.800000e-05,-3.133333e-05,5.476593,455.649522,14.636209,121.080691,333.756833,97.979767,14.871575,NaN,NaN,NaN
4,2023-10-23 17:10:46,111.9,81.300003,29.299999,Monday,14.637892,121.078200,En Route,-4.150000e-05,-3.533333e-05,6.060577,450.172928,14.636209,121.080691,334.373291,96.802118,20.932152,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567181,2023-11-14 00:56:36,107.0,76.000000,29.799999,Tuesday,14.633071,121.074693,En Route,-3.166667e-06,-1.666667e-07,0.352605,2.161636,NaN,NaN,NaN,0.203152,1062.558539,14.640187,121.07852,898.481995
567182,2023-11-14 00:56:37,106.6,76.000000,29.799999,Tuesday,14.633068,121.074693,En Route,-2.666667e-06,-1.666667e-07,0.297098,1.809032,NaN,NaN,NaN,0.170014,1062.855637,14.640187,121.07852,898.751940
567183,2023-11-14 00:56:38,106.1,76.000000,29.799999,Tuesday,14.633065,121.074694,En Route,-3.500000e-06,8.333333e-07,0.400061,1.511933,NaN,NaN,NaN,0.142092,1063.255699,14.640187,121.07852,899.050905
567184,2023-11-14 00:56:39,105.6,76.000000,29.799999,Tuesday,14.633058,121.074694,En Route,-6.500000e-06,5.000000e-07,0.724902,1.111872,NaN,NaN,NaN,0.104494,1063.980601,14.640187,121.07852,899.661406
